In [86]:
.ORIG x3000
    .SET MODE TERMINAL
        ST     R7, REG7
        JSR PUSH 
        ADD R5, R5, #4   ; R5 stores the number of min steps
        LD R1, INITIAL
        AND R0, R0, #0
        LEA R0, GAMEINTRO
        TRAP x22
LOOP:   LEA R0, FEEDBACK
        TRAP x22
        AND R0, R0, #0
        JSR DISP
        TRAP x24
        JSR GETCHOICE
        TRAP x22
        JSR GETPROG
        AND R0, R0, #0
        LEA R0, PIC
        TERMINAL R0, 0
        AND R0, #0
        TRAP x23
        JSR GETSTEP
        ST R1, NUM
        ADD R5, R5, #0
        BRp LOOP
        JSR POP        ;restore the value for all registers
        LD  R7, REG7
        HALT

INITIAL: .FILL #26 ;store the initial number
FEEDBACK: .STRINGZ "Number is: "
ASKFORCHOICE: .STRINGZ "\nEnter your choice:\n"
GAMEOPS: .STRINGZ "1) -1 2) /2 (A%2==0) 3) /3 (A%3==0)\n" 
REG7_STACK: .BLKW #10
REG7_TOP: .FILL REG7_STACK
NUM: .FILL #26  ; store updated number
PUSH:  ST R6, REG6     ;; save R5, R6 to MEMORY
       ST R5, REG5     ;; R7 has return address
                       ;; REG7 is in MEMORY already
                       ;; R6 and R5 are destroyed
       LD R6, SREG     ;; R6 is now TOP of stack
       STR R0, R6, #0      ;; push R0 on stack
       ADD R6, R6, #1  ;; incr
       STR R1, R6, #0      ;; push R1 on stack
       ADD R6, R6, #1  ;; incr
       STR R2, R6, #0      ;; push R2 on stack
       ADD R6, R6, #1  ;; incr
       STR R3, R6, #0      ;; push R3 on stack
       ADD R6, R6, #1  ;; incr
       STR R4, R6, #0      ;; push R4 on stack
       ADD R6, R6, #1  ;; incr
       STR R5, R6, #0      ;; push R5 on stack
       ADD R6, R6, #1  ;; incr
       LD R5, REG6     ;; get R6 from MEMORY (destroys R5)
       STR R5, R6, #0      ;; push R6 on stack
       ADD R6, R6, #1  ;; incr
       LD R5, REG7     ;; get R7 from MEMORY
       STR R5, R6, #0  ;; push R7 on stack
       ADD R6, R6, #1  ;; incr
       ST R6, SREG      ;; save incremented SREG
       LD R6, REG6
       LD R5, REG5     ;;
       RET
     
POP:   LD R6, SREG     ;; R6 is TOP of stack
       ADD R6, R6, #-1     ;; decr
       LDR  R1, R6, #0      ;; load R7
       ST  R1, REG7    ;; save to MEMORY
       ADD R6, R6, #-1     ;; decr
       LDR  R1, R6, #0      ;; load R6
       ST  R1, REG6    ;; save to MEMORY
       ADD R6, R6, #-1     ;; decr
       LDR  R5, R6, #0      ;; load R5
       ADD R6, R6, #-1     ;; decr
       LDR  R4, R6, #0      ;; load R4
       ADD R6, R6, #-1     ;; decr
       LDR  R3, R6, #0      ;; load R3
       ADD R6, R6, #-1     ;; decr
       LDR  R2, R6, #0      ;; load R2
       ADD R6, R6, #-1     ;; decr
       LDR  R1, R6, #0      ;; load R1
       ADD R6, R6, #-1     ;; decr
       LDR  R0, R6, #0      ;; load R0
       ST R6, SREG      ;; save decremented TOP
       LD R6, REG6, #0     ;; put R6 back the way it was
       RET


REG:    .BLKW    #8
SREG:   .FILL REG
REG5:   .FILL    #0
REG6:   .FILL    #0
REG7:   .FILL    #0
HALT     
  
GETCHOICE: AND R0, #0
           LEA R0, ASKFORCHOICE
           RET
           
;;R1 -- UPDATE THE RESULT
;;R5 -- STEP LIMIT
;;R6 -- ADDRESS IN R7(return address)
;;R2 -- GOAL

GETSTEP: ;;;;;;;;;;;;;;;;;;;;start pushing
         LD R6, REG7_TOP
         STR R7, R6, #0
         ;;;;;;;;;;;;;;;;;;;;Finish pushing R7 on stack
         ADD R0, R0, #-15
         ADD R0, R0, #-15
         ADD R0, R0, #-15
         ADD R0, R0, #-4
         BRz OPERATION1
         ADD R0, R0, #-1
         BRz OPERATION2
         ADD R0, R0, #-1
         BRz OPERATION3
         ;;;;;;;;;;;;;;;;;;;; start poping
         ADD R6, R6, #-1
         LDR R7, R6, #0
         ;;;;;;;;;;;;;;;;;;;;Finish poping
         RET

GETPROG:;;;;;;;;;;;;;;;;; start stack
         ADD R6, R6, #1
         STR R7, R6, #0
         ;;;;;;;;;;;;;;;;; Finish pushing R7 on stack 
         LD R2, INITIAL
         NOT R1, R1
         ADD R1, R1, #1
         ADD R3, R1, R2
         ADD R1, R1, #-1
         NOT R1, R1
         JSR MULT
         JSR DIV
         ADD R0, R0, #15
         ADD R0, R0, #15
         ADD R0, R0, #15
         ADD R0, R0, #3
         LD R3, PICUPDATE
         STR R0, R3, #0
         ;;;;;;;;;;;;;;;;;;;; start poping
         LDR R7, R6, #0
         ;;;;;;;;;;;;;;;;;;;;Finish poping
         RET

GAMEINTRO: .STRINGZ "Challenge: reduce A to B in min steps\n"
PIC: .STRINGZ "<img width='400' src='https://raw.githubusercontent.com/EileenFeng/240FinalProject/master/0.jpg'>" 
PICUPDATE: .FILL x314E

DISP: AND R0, R0, #0
      LD R0, TEMP
      LD R3, NUM
TEN:  ADD R3, R3, #-10
      BRn ONE
      ADD R0, R0, #1
      ADD R3, R3, #0
      BRp TEN
ONE:  ADD R3, R3, #10
      BRp #0
CALC: ADD R3, R3, #-1
      BRn #3
      LD R4, SEC
      ADD R0, R0, R4
      ADD R3, R3, #0
      BRp CALC
      ST R0, NUMUP
      LEA R0, NUMUP
      RET
      
NUMUP: .FILL #0
TERMIN: .FILL x0000
TEMP: .FILL x3030
SEC: .FILL x0100

MULT: AND R0, R0, #0
      ADD R0, R0, #9
      ADD R4, R3, #0
      ADD R3, R3, R4
      ADD R0, R0, #-1
      BRp #-3 
      RET

DIV: LD R4, GOAL
     NOT R4, R4
     ADD R4, R4, #1
     ADD R4, R2, R4
     NOT R4, R4
     ADD R4, R4, #1
     AND R0, R0, #0
     ADD R3, R3, R4
     BRn #2
     ADD R0, R0, #1
     BRp #-4
     RET

OPERATION1: ;;;;;;;;;;;;;;;;; operations(need to change @Eileen)
            ADD R1, R1, #-1
            ;;;;;;;;;;;;;;;;; start stack
            ADD R6, R6, #1
            STR R7, R6, #0
            ;;;;;;;;;;;;;;;;; Finish pushing R7 on stack
            JSR CHECKSTEP
            ADD R6, R6, #-1
            LDR R7, R6, #0
            ;;;;;;;;;;;;;;;;; Finish poping 
            RET

OPERATION2: ;;;;;;;;;;;;;;;;; start stack
            ADD R6, R6, #1
            STR R7, R6, #0
            ;;;;;;;;;;;;;;;;; Finish pushing R7 on stack
            JSR DIVTWO
            JSR CHECKSTEP
            ADD R6, R6, #-1
            LDR R7, R6, #0
            ;;;;;;;;;;;;;;;;; Finish poping 
            RET

DIVTWO: AND R2, R2, #0
        AND R3, R3, #0
        AND R4, R4, #0
        ADD R2, R2, #-2  ; ALWAYS USE R2 TO STORE THE DENOMITOR
        ADD R3, R3, R1   ; R3 to store the current R1
        ADD R4, R4, #0   ; R4 to store the result of calculating
        ADD R1, R1, R2   ; minus R1 by 2
        BRn #4
        BRz #9
        ADD R4, R4, #1
        ADD R1, R1, #0
        BRp #-6
        AND R1, R1, #0
        ADD R1, R1, R3  ; restore the value of R1 if not dividable
        AND R2, R2, #0
        AND R3, R3, #0
        AND R4, R4, #0
        RET
        ADD R1, R1, R4  ; otherwise udpate the value of R1
        ADD R1, R1, #1
        AND R2, R2, #0
        AND R3, R3, #0
        AND R4, R4, #0
        RET

OPERATION3: ;;;;;;;;;;;;;;;;; start stack
            ADD R6, R6, #1
            STR R7, R6, #0
            ;;;;;;;;;;;;;;;;; Finish pushing R7 on stack
            JSR DIVTHREE
            JSR CHECKSTEP
            ADD R6, R6, #-1
            LDR R7, R6, #0
            ;;;;;;;;;;;;;;;;; Finish poping 
            RET

DIVTHREE:   AND R2, R2, #0
            AND R3, R3, #0
            AND R4, R4, #0
            ADD R2, R2, #-3  ; ALWAYS USE R2 TO STORE THE DENOMITOR
            ADD R3, R3, R1   ; R3 to store the current R1
            ADD R4, R4, #0   ; R4 to store the result of calculating
            ADD R1, R1, R2   ; minus R1 by 2
            BRn #4
            BRz #9
            ADD R4, R4, #1
            ADD R1, R1, #0
            BRp #-6
            AND R1, R1, #0
            ADD R1, R1, R3  ; restore the value of R1 if not dividable
            AND R2, R2, #0
            AND R3, R3, #0
            AND R4, R4, #0
            RET
            ADD R1, R1, R4  ; otherwise udpate the value of R1
            ADD R1, R1, #1
            AND R2, R2, #0
            AND R3, R3, #0
            AND R4, R4, #0
            RET
            
CHECKSTEP: ;;;;;;;;;;;;;;;;;; update step
           ADD R6, R6, #1
           ADD R5, R5, #-1
           BRp #5
           ;;;;;;;;;;;;;;;;;; start stack
           STR R7, R6, #0
           ;;;;;;;;;;;;;;;;;; finish pushing R7
           ADD R5, R5, #0
           BRz FINALSTATE
           ;;;;;;;;;;;;;;;;;; start poping
           ADD R6, R6, #-1
           LDR R7, R6, #0
           ;;;;;;;;;;;;;;;;;; FINISH POPING
           RET

FINALSTATE:;;;;;;;;;;;;;;;;;; when step limit is consumed 
           ;;;;;;;;;;;;;;;;; start stack
           ADD R6, R6, #1
           STR R7, R6, #0
           ;;;;;;;;;;;;;;;;;; finish pushing R7
           LD R2, GOAL
           NOT R2, R2
           ADD R2, R2, #1
           ADD R3, R1, R2
           BRz WIN_STATE
           BRz #3 ; if not lose jump over the lose state
           AND R0, #0     
           LEA R0, LOSE_M  ; lose state
           TRAP x22
           ADD R6, R6, #-1
           LDR R7, R6, #0
           ;;;;;;;;;;;;;;;;;; FINISH POPING
           RET

WIN_STATE: ;;;;;;;;;;;;;;;;; start stack
           ADD R6, R6, #1
           STR R7, R6, #0
           ;;;;;;;;;;;;;;;;;; finish pushing R7
           JSR WIN_MES
           TRAP x22
           ADD R6, R6, #-1
           LDR R7, R6, #0
           ;;;;;;;;;;;;;;;;;; FINISH POPING
           ;;;;;;;;;;;;;;;;; start stack
           ADD R6, R6, #1
           STR R7, R6, #0
           ;;;;;;;;;;;;;;;;;; finish pushing R7
           AND R0, #0
           JSR WIN_PIC
           TRAP x22
           ;ADD R6, R6, #-1
           LDR R7, R6, #0
           ;;;;;;;;;;;;;;;;;; FINISH POPING
           HALT
    
WIN_MES: AND R0, #0
         LEA R0, WIN_M
         RET
         
WIN_PIC: AND R0, #0
         LEA R0, HEADER
         TERMINAL R0, 0
         RET
GOAL: .FILL #3
LOSE_M: .STRINGZ "SORRY YOU LOSE!\n"
WIN_M: .STRINGZ "\nYOU WIN!"
HEADER: .STRINGZ "<img width = '600' src='https://raw.githubusercontent.com/EileenFeng/240FinalProject/master/a.jpg'>"    
.END 

Assembled! Use %dis or %dump to examine; use %exe to run.


In [87]:
%exe

Challenge: reduce A to B in min steps
Number is: 26
Enter your choice:



Input a character> 2
2
Number is: 13
Enter your choice:



Input a character> 1
1
Number is: 12
Enter your choice:



Input a character> 2
2
Number is: 06
Enter your choice:



Input a character> 2
2

YOU WIN!

<img width = '600' src='https://raw.githubusercontent.com/EileenFeng/240FinalProject/master/a.jpg'>============================================================
Computation completed
Instructions: 5325
Cycles: 42219 (0.021109 milliseconds)

Registers:
PC: x048E
N: 0 Z: 0 P: 1 
R0: x320D R1: x0003 R2: xFFFD R3: x0000 
R4: x0000 R5: x0000 R6: x3066 R7: x31EA 
